# 1 Brand

In [23]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

def grab_brand_cars_php(brand) :
    session = requests.Session()
    my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"}
    url = 'https://www.classictrends.eu/fr/{}/'.format(brand)
    response = session.get(url, headers=my_headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')

    links = html_soup.find_all("a", href=True)

    car_list_php = []
    for i in links :
        if "php" in str(i) and "user" not in str(i) and "propos" not in str(i):
            car_list_php.append(i["href"])
    car_list_php
    return car_list_php

def grab_data(url_name) :
    session = requests.Session()
    my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"}
    url = 'https://www.classictrends.eu/fr/nissan/{}'.format(url_name)
    response = session.get(url, headers=my_headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')

    price_search = html_soup.find_all(class_="cote_box_cote")
    evo_price_search = html_soup.find_all(class_="evo_tab")
    name_search = html_soup.find_all(class_="auto_present_name")

    name = name_search[0].contents[0].strip()
    price = "".join(filter(str.isdigit, price_search[0].contents[0].strip()))
    
    if len(price) == 0 :
        price = '--'

    date_list = ['5 ans','4 ans','3 ans','2 ans','1 an','6 mois','3 mois']
    price_list = ['--','--','--','--','--','--','--']

    # date_list.append(i("div")[1].contents[1].string.replace("\n","").replace("\t","")) # Evolution

    for i in evo_price_search :
        if i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "5 ans" :                            # Date
            price_list[0] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "4 ans" :                          # Date
            price_list[1] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "3 ans" :                          # Date
            price_list[2] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "2 ans" :                          # Date
            price_list[3] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "1 an" :                           # Date
            price_list[4] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "6 mois" :                         # Date
            price_list[5] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "3 mois" :                         # Date
            price_list[6] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
    return name, price, price_list


brand = input("Brand : ")
car_list_php = grab_brand_cars_php("{}".format(brand))


name_price_list = []
for cars_php in car_list_php :
    if 'tools' not in cars_php :
        car = []
        name = None
        price = None
        name, price, price_list = grab_data(cars_php)
        car.append(name)
        car.append(price)
        car.append(price_list)
        name_price_list.append(car)
    time.sleep(0.5)

name_list_pd = []
price_list_pd = []
price_list_5a_pd = []
price_list_4a_pd = []
price_list_3a_pd = []
price_list_2a_pd = []
price_list_1a_pd = []
price_list_6m_pd = []
price_list_3m_pd = []

for car in name_price_list :
    name_list_pd.append(car[0])                     # Nom
    price_list_pd.append(car[1])                    # Prix actuel
    price_list_5a_pd.append(car[2][0])              # Prix old
    price_list_4a_pd.append(car[2][1])              # Prix old
    price_list_3a_pd.append(car[2][2])              # Prix old
    price_list_2a_pd.append(car[2][3])              # Prix old
    price_list_1a_pd.append(car[2][4])              # Prix old
    price_list_6m_pd.append(car[2][5])              # Prix old
    price_list_3m_pd.append(car[2][6])              # Prix old

d = {'Name': name_list_pd, 'Price now': price_list_pd, '5 ans': price_list_5a_pd, '4 ans': price_list_4a_pd, '3 ans': price_list_3a_pd, '2 ans': price_list_2a_pd, '1 ans': price_list_1a_pd, '6 mois': price_list_6m_pd, '3 mois': price_list_3m_pd}
df = pd.DataFrame(data=d)
df.to_excel("{}.xlsx".format(brand))
df

KeyboardInterrupt: 

# Every Brand - multiple doc

In [21]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

def grab_brand_cars_php(brand) :
    session = requests.Session()
    my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"}
    url = 'https://www.classictrends.eu/fr/{}/'.format(brand)
    response = session.get(url, headers=my_headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')

    links = html_soup.find_all("a", href=True)

    car_list_php = []
    for i in links :
        if "php" in str(i) and "user" not in str(i) and "propos" not in str(i):
            car_list_php.append(i["href"])
    car_list_php
    return car_list_php

def grab_data(url_name) :
    session = requests.Session()
    my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"}
    url = 'https://www.classictrends.eu/fr/nissan/{}'.format(url_name)
    response = session.get(url, headers=my_headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')

    price_search = html_soup.find_all(class_="cote_box_cote")
    evo_price_search = html_soup.find_all(class_="evo_tab")
    name_search = html_soup.find_all(class_="auto_present_name")

    name = name_search[0].contents[0].strip()
    price = "".join(filter(str.isdigit, price_search[0].contents[0].strip()))
    
    if len(price) == 0 :
        price = '--'

    date_list = ['5 ans','4 ans','3 ans','2 ans','1 an','6 mois','3 mois']
    price_list = ['--','--','--','--','--','--','--']

    # date_list.append(i("div")[1].contents[1].string.replace("\n","").replace("\t","")) # Evolution

    for i in evo_price_search :
        if i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "5 ans" :                            # Date
            price_list[0] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "4 ans" :                          # Date
            price_list[1] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "3 ans" :                          # Date
            price_list[2] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "2 ans" :                          # Date
            price_list[3] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "1 an" :                           # Date
            price_list[4] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "6 mois" :                         # Date
            price_list[5] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "3 mois" :                         # Date
            price_list[6] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
    return name, price, price_list


brands = ["DS","Dacia","Panhard","Talbot","Facel-Vega","Infiniti","Matra","Mclaren","Tesla","Austin","Alpine","Autobianchi","Venturi","Datsun","Hummer","Bugatti","Innocenti","Jensen","Lada","PGO","Ac","Delorean","Mega","Mercury","Piaggio","Velam"]
# "Lexus","Saab","Maserati","Kia","Lotus","Chrysler","Skoda","Triumph","Jeep","Land-rover","MG","Bentley","Simca","Smart","Hyundai","Dodge","Rolls-royce","Subaru","Tvr","Lamborghini",
# "Alfa-Romeo","Nissan","Opel","Toyota","Citroen","Honda","Volvo","Mazda","Mitsubishi","Ferrari","Lancia","Mini","Jaguar","Cadillac","Suzuki","Seat","Aston-Martin","Chevrolet",
# "Porsche","Renault","Volkswagen","Mercedes","Peugeot","BMW","Audi","Ford","Fiat",

for brand in brands :
    car_list_php = grab_brand_cars_php("{}".format(brand))

    name_price_list = []
    for cars_php in car_list_php :
        if 'tools' not in cars_php :
            car = []
            name = None
            price = None
            name, price, price_list = grab_data(cars_php)
            car.append(name)
            car.append(price)
            car.append(price_list)
            name_price_list.append(car)
        time.sleep(0.5)

    name_list_pd = []
    price_list_pd = []
    price_list_5a_pd = []
    price_list_4a_pd = []
    price_list_3a_pd = []
    price_list_2a_pd = []
    price_list_1a_pd = []
    price_list_6m_pd = []
    price_list_3m_pd = []

    for car in name_price_list :
        name_list_pd.append(car[0])                     # Nom
        price_list_pd.append(car[1])                    # Prix actuel
        price_list_5a_pd.append(car[2][0])              # Prix old
        price_list_4a_pd.append(car[2][1])              # Prix old
        price_list_3a_pd.append(car[2][2])              # Prix old
        price_list_2a_pd.append(car[2][3])              # Prix old
        price_list_1a_pd.append(car[2][4])              # Prix old
        price_list_6m_pd.append(car[2][5])              # Prix old
        price_list_3m_pd.append(car[2][6])              # Prix old

    d = {'Name': name_list_pd, 'Price now': price_list_pd, '5 ans': price_list_5a_pd, '4 ans': price_list_4a_pd, '3 ans': price_list_3a_pd, '2 ans': price_list_2a_pd, '1 ans': price_list_1a_pd, '6 mois': price_list_6m_pd, '3 mois': price_list_3m_pd}
    df = pd.DataFrame(data=d)
    df.to_excel("{}.xlsx".format(brand))
    df

# Every Brand - 1 doc

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd

def grab_brand_cars_php(brand) :
    session = requests.Session()
    my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"}
    url = 'https://www.classictrends.eu/fr/{}/'.format(brand)
    response = session.get(url, headers=my_headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')

    links = html_soup.find_all("a", href=True)

    car_list_php = []
    for i in links :
        if "php" in str(i) and "user" not in str(i) and "propos" not in str(i):
            car_list_php.append(i["href"])
    car_list_php
    return car_list_php

def grab_data(url_name) :
    session = requests.Session()
    my_headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36","Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9"}
    url = 'https://www.classictrends.eu/fr/nissan/{}'.format(url_name)
    response = session.get(url, headers=my_headers)
    html_soup = BeautifulSoup(response.text, 'html.parser')

    price_search = html_soup.find_all(class_="cote_box_cote")
    evo_price_search = html_soup.find_all(class_="evo_tab")
    name_search = html_soup.find_all(class_="auto_present_name")

    name = name_search[0].contents[0].strip()
    price = "".join(filter(str.isdigit, price_search[0].contents[0].strip()))
    
    if len(price) == 0 :
        price = '--'

    date_list = ['5 ans','4 ans','3 ans','2 ans','1 an','6 mois','3 mois']
    price_list = ['--','--','--','--','--','--','--']

    # date_list.append(i("div")[1].contents[1].string.replace("\n","").replace("\t","")) # Evolution

    for i in evo_price_search :
        if i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "5 ans" :                            # Date
            price_list[0] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "4 ans" :                          # Date
            price_list[1] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "3 ans" :                          # Date
            price_list[2] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "2 ans" :                          # Date
            price_list[3] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "1 an" :                           # Date
            price_list[4] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "6 mois" :                         # Date
            price_list[5] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
        elif i("div")[0].contents[3].string.replace("\n","").replace("\t","") == "3 mois" :                         # Date
            price_list[6] = "".join(filter(str.isdigit, i("div")[1].contents[4].string.replace("(","").replace(")","").replace('\xa0', ' ')))         # Prix
    return name, price, price_list


brands = ["Porsche","Renault","Volkswagen","Mercedes","Peugeot","BMW","Audi","Ford","Fiat","Alfa-Romeo","Nissan","Opel","Toyota","Citroen","Honda","Volvo","Mazda","Mitsubishi","Ferrari","Lancia","Mini","Jaguar","Cadillac","Suzuki","Seat","Aston-Martin","Chevrolet","Lexus","Saab","Maserati","Kia","Lotus","Chrysler","Skoda","Triumph","Jeep","Land-rover","MG","Bentley","Simca","Smart","Hyundai","Dodge","Rolls-royce","Subaru","Tvr","Lamborghini","DS","Dacia","Panhard","Talbot","Facel-Vega","Infiniti","Matra","Mclaren","Tesla","Austin","Alpine","Autobianchi","Venturi","Datsun","Hummer","Bugatti","Innocenti","Jensen","Lada","PGO","Ac","Delorean","Mega","Mercury","Piaggio","Velam"]


car_list_php = grab_brand_cars_php("{}".format(brand))

name_price_list = []
for cars_php in car_list_php :
    car = []
    name = None
    price = None
    name, price, price_list = grab_data(cars_php)
    car.append(name)
    car.append(price)
    car.append(price_list)
    name_price_list.append(car)
    time.sleep(0.5)

name_list_pd = []
price_list_pd = []
price_list_5a_pd = []
price_list_4a_pd = []
price_list_3a_pd = []
price_list_2a_pd = []
price_list_1a_pd = []
price_list_6m_pd = []
price_list_3m_pd = []

for car in name_price_list :
    name_list_pd.append(car[0])                     # Nom
    price_list_pd.append(car[1])                    # Prix actuel
    price_list_5a_pd.append(car[2][0])              # Prix old
    price_list_4a_pd.append(car[2][1])              # Prix old
    price_list_3a_pd.append(car[2][2])              # Prix old
    price_list_2a_pd.append(car[2][3])              # Prix old
    price_list_1a_pd.append(car[2][4])              # Prix old
    price_list_6m_pd.append(car[2][5])              # Prix old
    price_list_3m_pd.append(car[2][6])              # Prix old

d = {'Name': name_list_pd, 'Price now': price_list_pd, '5 ans': price_list_5a_pd, '4 ans': price_list_4a_pd, '3 ans': price_list_3a_pd, '2 ans': price_list_2a_pd, '1 ans': price_list_1a_pd, '6 mois': price_list_6m_pd, '3 mois': price_list_3m_pd}
df = pd.DataFrame(data=d)
df.to_excel("{}.xlsx".format(brand))
df

In [ ]:
brands = ["Porsche","Renault","Volkswagen","Mercedes","Peugeot","BMW","Audi","Ford","Fiat","Alfa-Romeo","Nissan","Opel","Toyota","Citroen","Honda","Volvo","Mazda","Mitsubishi","Ferrari","Lancia","Mini","Jaguar","Cadillac","Suzuki","Seat","Aston-Martin","Chevrolet","Lexus","Saab","Maserati","Kia","Lotus","Chrysler","Skoda","Triumph","Jeep","Land-rover","MG","Bentley","Simca","Smart","Hyundai","Dodge","Rolls-royce","Subaru","Tvr","Lamborghini","DS","Dacia","Panhard","Talbot","Facel-Vega","Infiniti","Matra","Mclaren","Tesla","Austin","Alpine","Autobianchi","Venturi","Datsun","Hummer","Bugatti","Innocenti","Jensen","Lada","PGO","Ac","Delorean","Mega","Mercury","Piaggio","Velam"]